In [44]:
from io import BytesIO
import time
import imageio
from PIL import Image
import numpy as np
import random

In [2]:
def raw_to_image(img_size, bytes_list):
    return Image.frombytes("RGB", img_size, bytes(bytes_list))

In [8]:
def get_noise(base_size, num_repeats):
    """
    now obsolete, the noise generation is part of the algorithm
    """
    
    noise_data = [255*random.getrandbits(1) for x in range(base_size[0]*base_size[1])]
    noise_chunk = raw_to_image(base_size, noise_data)
    #noise_chunk.save("noise_chunk.png")

    noise = Image.new("L", size)
    x_offset = 0
    for _ in range(num_repeats):
        noise.paste(noise_chunk, (x_offset, 0))
        x_offset += noise_chunk.size[0]
    #noise.save("full_noise.png")
    return noise

In [65]:
def make_stereogram(size, raw):
    """
    honestly no idea what's happening here, but it works
    https://www.cs.waikato.ac.nz/~ihw/papers/94-HWT-SI-IHW-SIRDS-paper.pdf
    """ 

    target_data = np.ndarray((size[1], size[0], 3), dtype=np.uint8, order="C") # flip width and height? C or F?

    for y, row in enumerate(raw):
        #print(y)

        same = list(range(size[0]))

        for x in range(len(row)):
            s = int((1-mu*row[x])*E/(2-mu*row[x]))
            
            left = x-s//2
            right = left+s

            if left < 0 or right >= size[0]:
                continue

            visible = True
            t = 1
            zt = 0

            while visible and zt < 1:
                zt = row[x] + 2*(2 - mu*row[x])*t/(mu*E)
                visible = row[x-t] < zt and row[x+t] < zt
                t += 1

            if visible:
                l = same[left]
                while l != left and l != right:
                    if l < right:
                        left = l
                        l = same[left]
                    else:
                        same[left] = right
                        left = right
                        l = same[left]
                        right = l
                
                same[left] = right

        pix = [0] * size[0]
        for x in reversed(range(len(row))):
            if same[x] == x:
                pix[x] = 255*random.getrandbits(1)
            else:
                pix[x] = pix[same[x]]
            
            val = pix[x]
            target_data[y][x] = [val, val, val] # flip x and y?

    far = int(E/2)
    side = size[1]//20
    target_data[size[1]-side:, size[0]//2-far//2-side//2:size[0]//2-far//2+side//2] = [0, 0, 0]
    target_data[size[1]-side:, size[0]//2+far//2-side//2:size[0]//2+far//2+side//2] = [0, 0, 0]
    return target_data

In [60]:
total_size = (1920, 1080)

mu = 1/3
DPI = 72
E = 6*DPI

all basic preparation now done

In [55]:
#noise_base = get_noise(noise_size, repeat_chunks)
src = Image.open("source.png").convert("L")
src_raw = list(src.getdata(0))
src_raw = [[j/255 for j in src_raw[i:i + total_size[0]]] for i in range(0, len(src_raw), total_size[0])]

In [15]:
with open("bad_apple_better.mkv", "rb") as file:
    content = file.read()
    video = imageio.get_reader(BytesIO(content), 'ffmpeg')

print(video.get_meta_data())

{'plugin': 'ffmpeg', 'nframes': inf, 'ffmpeg_version': '4.2.2-static https://johnvansickle.com/ffmpeg/ built with gcc 8 (Debian 8.3.0-6)', 'codec': 'h264', 'pix_fmt': 'yuv420p(progressive)', 'fps': 60.0, 'source_size': (2566, 1924), 'size': (2566, 1924), 'duration': 219.16}


In [67]:
writer = imageio.get_writer("target.mkv", fps=3)
src_size = video.get_meta_data()["size"]

for index, img_src in enumerate(video.iter_data()):
    if index % 20 != 0:
        continue
    
    if index < 900:
        continue
    if index >= 1100:
        break

    print(index, end=" ")
    t0 = time.time()

    src_raw = np.asarray(img_src)[:, :, 0] / 255 # just taking the red channel, assuming grayscale anyway

    img_data = make_stereogram(src_size, src_raw)
    writer.append_data(img_data)

    print(time.time()-t0)

writer.close()

900 IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (2566, 1924) to (2576, 1936) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).
219.96780633926392
920 172.07382321357727
940 167.61366510391235
960 108.5531735420227
980 178.05480813980103
1000 156.4303388595581
1020 151.06680488586426
1040 165.51153779029846
1060 189.88237476348877
1080 240.814297914505


In [66]:
target = make_stereogram(total_size, src_raw)
Image.fromarray(target).save("target.png")